In [2]:
%pip install langchain
%pip install langchain-community
%pip install langchain-huggingface
%pip install langchainhub
%pip install langchain_chroma 

# Importando a biblioteca torch
import torch

# Importando a biblioteca os para interação com o sistema operacional
import os

# Instalando a versão compatível do pacote tokenizers
%pip install "tokenizers==0.20.3"

# Instalando a versão compatível do pacote transformers
%pip install "transformers==4.46.3"

# Instalando a versão compatível do pacote chromadb
%pip install "chromadb==0.5.23"

# Importando funções e classes da biblioteca transformers para manipulação de modelos de linguagem
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

# Importando o HuggingFacePipeline do langchain_huggingface para integrar o Hugging Face com o Langchain
from langchain_huggingface import HuggingFacePipeline

# Importando a classe PromptTemplate do Langchain para criar templates de prompts
from langchain import PromptTemplate

# Importando classes adicionais para manipulação de mensagens e prompts de conversa
from langchain_core.prompts import (
    ChatPromptTemplate,         # Template para chats
    HumanMessagePromptTemplate, # Template para mensagens humanas
    MessagesPlaceholder        # Placeholder para mensagens
)

# Importando a classe SystemMessage para representar mensagens do sistema
from langchain_core.messages import SystemMessage

# Importando StrOutputParser para definir a forma de análise de saída de texto
from langchain_core.output_parsers import StrOutputParser

# Importando RunnablePassthrough para criar um comportamento de "passagem" em processos Langchain
from langchain_core.runnables import RunnablePassthrough


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)

llm = HuggingFacePipeline(pipeline=pipe)

In [6]:
input = "Quem foi a primeira pessoa no espaço?"

output = llm.invoke(input)
print(output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 Yuri Gagarin
Yuri Alekseyevich Gagarin (1 de março de 1934 – 27 de março de 1968) foi um cosmonauta soviético e o primeiro ser humano a viajar ao espaço. Ele se tornou uma figura icônica da União Soviética, sendo considerado um herói nacional.
Gagarin nasceu em Klushino, Oblast de Smolensk, Rússia, em uma família de agricultores. Ele estudou na escola local antes de ingressar na Escola Técnica de Astronáutica em 1950. Após concluir seus estudos, ele trabalhou como engenheiro mecânico em uma fábrica de motores em Kaluga.
Em 1960, Gagarin foi selecionado para o programa espacial soviético, juntamente com outros candidatos. Ele passou por treinamento rigoroso, incluindo simulações de voo e exercícios físicos intensos. Em 12 de abril de 1961, Gagarin foi nomeado para voar no Vostok 1, o primeiro veículo espacial projetado para transportar humanos.
O Vostok 1 foi lançado do Cosmódromo de Baikonur em 12 de abril de 1961 às 06h00 UTC. A bordo estava Gagarin, que completou uma órbita ao redor

In [7]:
# adequando o template
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header|>assistant<|end_header_id|>
"""

In [8]:
system_prompt = "Você é um assistente que sempre retorna em portugues"
user_prompt = input

In [9]:
prompt_template = template.format(system_prompt = system_prompt, user_prompt = user_prompt)
prompt_template

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente que sempre retorna em portugues\n<|eot_id|>\n<|start_header|>user<|end_header_id|>\nQuem foi a primeira pessoa no espaço?\n<|eot_id|>\n<|start_header|>assistant<|end_header_id|>\n'

In [10]:
output = llm.invoke(prompt_template)
print(output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


A primeira pessoa no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele realizou uma viagem ao espaço em 12 de abril de 1961, durante a missão Vostok 1. A bordo do veículo espacial Vostok 3KA-9, Gagarin completou uma órbita ao redor da Terra, permanecendo no espaço por cerca de 108 minutos.


In [11]:
from langchain_core.messages import (HumanMessage, SystemMessage)
from langchain_huggingface import ChatHuggingFace

In [12]:
msgs = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt)
]

In [13]:
chat_model = ChatHuggingFace(llm = llm)

In [14]:
model_template = tokenizer.chat_template
model_template

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interp

In [15]:
chat_model._to_chat_prompt(msgs)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nVocê é um assistente que sempre retorna em portugues<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuem foi a primeira pessoa no espaço?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [18]:
res = chat_model.invoke(msgs)
print(res)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


content='A primeira pessoa a viajar pelo espaço foi Yuri Gagarin, um cosmonauta soviético. Ele realizou uma órbita ao redor da Terra em 12 de abril de 1961, durante a missão Vostok 1. Gagarin ficou conhecido como o "Homem Espacial" e sua conquista histórica marcou um momento importante na exploração espacial.\n\nNo entanto, se você está perguntando sobre quem foi a primeira pessoa a caminhar no espaço, a resposta é Alan Shepard, um astronauta americano. Ele realizou uma caminhada no espaço em 2 de fevereiro de 1971, durante a missão Apollo 14.' additional_kwargs={} response_metadata={} id='run-fb153af7-7de5-4806-a53b-bb54d084c7f6-0'
